<a href="https://colab.research.google.com/github/fholken/curso_agentes_ai/blob/main/2_Chat_memory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
pip install -qU langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.9 MB/s eta 0:00:00


In [7]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


In [14]:
from langchain_openai import ChatOpenAI
from  langchain_core.messages import HumanMessage, AIMessage

model = ChatOpenAI(model="gpt-3.5-turbo")

In [12]:
model.invoke([HumanMessage(content="Hi!, My name is Mauricio")]).content

'Nice to meet you, Mauricio! How can I assist you today?'

Acá no puede encontrar nada, ya que no tiene un historial.

In [13]:
model.invoke([HumanMessage(content="What is my name?")]).content

"I'm sorry, I do not have that information."

In [15]:
model.invoke([
    HumanMessage(content="Hi!, My name is Mauricio"),
    AIMessage(content="Hi Mauricio! How can I help you today?"),
    HumanMessage(content="What is my name?")
]).content

'Your name is Mauricio.'

Importamos **BaseChatMessageHistory**: Para construir la funcion que va ir construyendo el historial

Importamos **InMemoryChatMessageHistory**: Para ir guardando en memoria

Importamos **RunnableWithMessageHistory**: Para construir las cadenas y tener el historial de la conversacion.

In [16]:
from langchain_core.chat_history import BaseChatMessageHistory, InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

Guardaremos en un diccionario

```
store = {}
```



In [17]:
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [18]:
with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [19]:
config = {'configurable': {'session_id': 'abc2'}}

In [20]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi!, My name is Mauricio")],
    config=config
)

In [21]:
response.content

'Hello Mauricio! How can I assist you today?'

In [22]:
response = with_message_history.invoke(
    [HumanMessage(content="What is my name")],
    config=config
)

In [23]:
response.content

'Your name is Mauricio.'

In [24]:
config = {'configurable': {'session_id': 'abc3'}}

In [25]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi!, My name is Daniela")],
    config=config
)

In [26]:
response.content

'Hello Daniela! How can I assist you today?'

In [27]:
store

{'abc2': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi!, My name is Mauricio', additional_kwargs={}, response_metadata={}), AIMessage(content='Hello Mauricio! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 14, 'total_tokens': 25, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-0bc2fe79-53b6-4891-a172-e56dfa26436b-0', usage_metadata={'input_tokens': 14, 'output_tokens': 11, 'total_tokens': 25, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}), HumanMessage(content='What is my name', additional_kwargs={}, response_metadata={}), AIMessage(content='You

# Implementación de memoria en cadenas

In [39]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a nice AI bot that answers questions"),
    MessagesPlaceholder(variable_name="messages"),
])

In [40]:
chain = prompt | model

In [41]:
question = "What is my name?"

response = chain.invoke({"messages": [
    HumanMessage(content="Hi!, My name is Mauricio")
]})

In [42]:
response.content

'Hello Mauricio! How can I assist you today?'

In [32]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [33]:
config = {'configurable': {'session_id': 'abc4'}}

In [34]:
response = with_message_history.invoke(
    [HumanMessage(content="What is my name")],
    config=config
)
response.content

"I'm sorry, but I am not able to know your name as I am an AI bot and do not have access to that information. How can I assist you today?"

In [35]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi!, My name is Santiago")],
    config=config
)
response.content

'Hello Santiago! How can I assist you today?'

In [36]:
store

{'abc2': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi!, My name is Mauricio', additional_kwargs={}, response_metadata={}), AIMessage(content='Hello Mauricio! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 14, 'total_tokens': 25, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-0bc2fe79-53b6-4891-a172-e56dfa26436b-0', usage_metadata={'input_tokens': 14, 'output_tokens': 11, 'total_tokens': 25, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}), HumanMessage(content='What is my name', additional_kwargs={}, response_metadata={}), AIMessage(content='You

In [43]:
# Bucle interactivo para preguntas dinámicas
def main():
    print("Bienvenido al chat interactivo. Escribe 'salir' para terminar.")

    while True:
        user_input = input("Tu pregunta: ")

        # Salir del bucle si el usuario escribe 'salir'
        if user_input.lower() == "salir":
            print("Cerrando sesión...")
            break

        # Crear un mensaje de usuario y procesarlo
        human_message = HumanMessage(content=user_input)

        try:
            # Obtener la respuesta del modelo
            response = with_message_history.invoke([human_message], config=config)

            # Mostrar la respuesta del modelo
            print(f"AI: {response.content}")

        except Exception as e:
            print(f"Error: {e}", file=sys.stderr)

if __name__ == "__main__":
    main()

Bienvenido al chat interactivo. Escribe 'salir' para terminar.
Tu pregunta: Cual es mi nombre?
AI: Tu nombre es Santiago. ¿En qué puedo ayudarte hoy, Santiago?
Tu pregunta: Cuantas vidas tiene un gato?
AI: Los gatos suelen tener 9 vidas en la cultura popular, pero en realidad los gatos tienen una sola vida, al igual que cualquier otro animal. La creencia de las 9 vidas se refiere a la capacidad de los gatos para sobrevivir a situaciones peligrosas y tener una gran agilidad. ¿Hay algo más en lo que pueda ayudarte?
Tu pregunta: Cual es mi nombre?
AI: Tu nombre es Santiago. ¿En qué más puedo ayudarte hoy?


KeyboardInterrupt: Interrupted by user